# Add yolo results to dataframe, compute od in original image, save images 

In [2]:
import pandas as pd
import os
from tqdm import tqdm
from collections import Counter
from src.preprocessing import crop_od, resize_square, make_square
import cv2

In [2]:
df = pd.read_csv("data/img_info.csv", index_col=0)
df

,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,orig_file_number,shuf_file_number,labels_string,labels_int
10009,DEV00000.jpg,SHUF03193,-72,-856,3176,3176,1.0,0,3193,NRG,0.0
14069,DEV00001.jpg,SHUF08836,-34,-32,1810,1810,1.0,1,8836,RG,1.0
5185,DEV00002.jpg,SHUF09765,196,-128,2336,2336,1.0,2,9765,NRG,0.0
5338,DEV00003.jpg,SHUF08574,310,-270,2925,2925,1.0,3,8574,NRG,0.0
13810,DEV00004.jpg,SHUF09120,-35,-35,1807,1807,1.0,4,9120,NRG,0.0
...,...,...,...,...,...,...,...,...,...,...,...
3467,DEV14995.jpg,SHUF02843,-69,-628,2176,2176,1.0,14995,2843,NRG,0.0
9059,DEV14996.jpg,SHUF10163,233,-246,2346,2346,1.0,14996,10163,RG,1.0
8883,DEV14997.jpg,SHUF07094,196,-128,2336,2336,1.0,14997,7094,NRG,0.0
125,DEV14998.jpg,SHUF01598,-34,-34,1660,1660,1.0,14998,1598,NRG,0.0


## Extract predictions and add to dataframe

In [3]:
leng = []

for filename in tqdm(os.listdir("data/labels")):
    predictions = open(f"data/labels/{filename}", "r").read()
    predictions = predictions.split("\n")[:-1]
    predictions = [pred.split() for pred in predictions]
    leng.append(len(predictions))

100%|██████████| 14942/14942 [00:36<00:00, 408.08it/s]


In [4]:
count = Counter(leng)
count # in 117 instances we only have 1 prediction 
# (I checked, its like half/half fovea or OD. so like 50 ODs are missing but in that case we can just use the full retina)

Counter({2: 13479, 3: 605, 1: 823, 4: 35})

In [5]:
len(os.listdir("data/labels"))

14942

In [6]:
for filename in tqdm(os.listdir("data/labels")):
    predictions = open(f"data/labels/{filename}", "r").read()
    predictions = predictions.split("\n")[:-1]
    predictions = [pred.split() for pred in predictions]
    predictions = [[float(number) for number in pred] for pred in predictions]
    for pred in predictions:
        max_od = 0
        max_fovea = 0
        if pred[0] == 0 and pred[-1] > max_od:
            df.loc[df.new_file == filename[:-4], "odc_x_ratio"] = pred[1]
            df.loc[df.new_file == filename[:-4], "odc_y_ratio"] = pred[2]
            df.loc[df.new_file == filename[:-4], "odc_width_ratio"] = pred[3]
            df.loc[df.new_file == filename[:-4], "odc_height_ratio"] = pred[4]
            df.loc[df.new_file == filename[:-4], "odc_height_conf"] = pred[5]
            max_od = pred[-1]
        if pred[0] == 1 and pred[-1] > max_fovea:
            df.loc[df.new_file == filename[:-4], "fovea_x_ratio"] = pred[1]
            df.loc[df.new_file == filename[:-4], "fovea_y_ratio"] = pred[2]
            df.loc[df.new_file == filename[:-4], "fovea_width_ratio"] = pred[3]
            df.loc[df.new_file == filename[:-4], "fovea_height_ratio"] = pred[4]
            df.loc[df.new_file == filename[:-4], "fovea_height_conf"] = pred[5]
            max_fovea = pred[-1]

df

100%|██████████| 14942/14942 [03:23<00:00, 73.48it/s]


,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,orig_file_number,shuf_file_number,labels_string,...,fovea_x_ratio,fovea_y_ratio,fovea_width_ratio,fovea_height_ratio,fovea_height_conf,odc_x_ratio,odc_y_ratio,odc_width_ratio,odc_height_ratio,odc_height_conf
10009,DEV00000.jpg,SHUF03193,-72,-856,3176,3176,1.0,0,3193,NRG,...,0.627344,0.510156,0.170312,0.170312,0.912301,0.274219,0.446875,0.185937,0.184375,0.938848
14069,DEV00001.jpg,SHUF08836,-34,-32,1810,1810,1.0,1,8836,RG,...,0.335156,0.511719,0.160938,0.157813,0.920892,0.718750,0.447656,0.159375,0.154687,0.949706
5185,DEV00002.jpg,SHUF09765,196,-128,2336,2336,1.0,2,9765,NRG,...,0.607813,0.496094,0.150000,0.148438,0.913449,0.318750,0.445312,0.156250,0.153125,0.949999
5338,DEV00003.jpg,SHUF08574,310,-270,2925,2925,1.0,3,8574,NRG,...,0.532031,0.384375,0.132812,0.131250,0.589760,0.240625,0.426562,0.131250,0.134375,0.591959
13810,DEV00004.jpg,SHUF09120,-35,-35,1807,1807,1.0,4,9120,NRG,...,0.676562,0.529688,0.156250,0.153125,0.919374,0.341406,0.471094,0.157813,0.154687,0.956209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,DEV14995.jpg,SHUF02843,-69,-628,2176,2176,1.0,14995,2843,NRG,...,0.328906,0.521875,0.179688,0.181250,0.924731,0.735937,0.492969,0.190625,0.192188,0.940419
9059,DEV14996.jpg,SHUF10163,233,-246,2346,2346,1.0,14996,10163,RG,...,NaN,NaN,NaN,NaN,NaN,0.660937,0.539062,0.121875,0.125000,0.626001
8883,DEV14997.jpg,SHUF07094,196,-128,2336,2336,1.0,14997,7094,NRG,...,0.497656,0.473437,0.164062,0.162500,0.916813,0.115625,0.453906,0.184375,0.185937,0.954872
125,DEV14998.jpg,SHUF01598,-34,-34,1660,1660,1.0,14998,1598,NRG,...,0.344531,0.510938,0.151562,0.153125,0.888888,0.706250,0.427344,0.159375,0.157813,0.939186


## Calculate OD size and center on original image

In [26]:
df["fovea_x_square"] = round(df.orig_crop_side * df.fovea_x_ratio, 0)
df["odc_x_square"] = round(df.orig_crop_side * df.odc_x_ratio, 0)
df["fovea_y_square"] = round(df.orig_crop_side * df.fovea_y_ratio, 0)
df["odc_y_square"] = round(df.orig_crop_side * df.odc_y_ratio, 0)
df["od_side_ratio_avg"] = (df.odc_height_ratio + df.odc_width_ratio)/2
df["odc_side_pxl"] = round(df.orig_crop_side * df.od_side_ratio_avg, 0)
df["odc_x_rect"] = df.odc_x_square - df.delta_y # it seems like delta x refers to how many pixels are cut of from the top and delta_y to the left
df["odc_y_rect"] = df.odc_y_square - df.delta_x

df

,orig_file,new_file,delta_x,delta_y,orig_crop_side,side,scaling,orig_file_number,shuf_file_number,labels_string,...,odc_height_conf,fovea_x_square,odc_x_square,fovea_y_square,odc_y_square,od_side_ratio_avg,odc_side_pxl,odc_x_rect,odc_y_rexr,odc_y_rect
10009,DEV00000.jpg,SHUF03193,-72,-856,3176,3176,1.0,0,3193,NRG,...,0.938848,1992.0,871.0,1620.0,1419.0,0.185156,588.0,1727.0,943.0,1491.0
14069,DEV00001.jpg,SHUF08836,-34,-32,1810,1810,1.0,1,8836,RG,...,0.949706,607.0,1301.0,926.0,810.0,0.157031,284.0,1333.0,1335.0,844.0
5185,DEV00002.jpg,SHUF09765,196,-128,2336,2336,1.0,2,9765,NRG,...,0.949999,1420.0,745.0,1159.0,1040.0,0.154688,361.0,873.0,549.0,844.0
5338,DEV00003.jpg,SHUF08574,310,-270,2925,2925,1.0,3,8574,NRG,...,0.591959,1556.0,704.0,1124.0,1248.0,0.132812,388.0,974.0,394.0,938.0
13810,DEV00004.jpg,SHUF09120,-35,-35,1807,1807,1.0,4,9120,NRG,...,0.956209,1223.0,617.0,957.0,851.0,0.156250,282.0,652.0,652.0,886.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3467,DEV14995.jpg,SHUF02843,-69,-628,2176,2176,1.0,14995,2843,NRG,...,0.940419,716.0,1601.0,1136.0,1073.0,0.191406,417.0,2229.0,1670.0,1142.0
9059,DEV14996.jpg,SHUF10163,233,-246,2346,2346,1.0,14996,10163,RG,...,0.626001,NaN,1551.0,NaN,1265.0,0.123438,290.0,1797.0,1318.0,1032.0
8883,DEV14997.jpg,SHUF07094,196,-128,2336,2336,1.0,14997,7094,NRG,...,0.954872,1163.0,270.0,1106.0,1060.0,0.185156,433.0,398.0,74.0,864.0
125,DEV14998.jpg,SHUF01598,-34,-34,1660,1660,1.0,14998,1598,NRG,...,0.939186,572.0,1172.0,848.0,709.0,0.158594,263.0,1206.0,1206.0,743.0


In [27]:
df.to_csv("data/img_info_extended.csv")

In [3]:
df = pd.read_csv("data/img_info_extended.csv", index_col=0)

## Crop original images

In [4]:
FACTOR = 3
THRESHOLD = 20

for filename in tqdm(os.listdir("data/cfp")):
    img = cv2.imread(f"data/cfp/{filename}")
    img_data = df.loc[df.orig_file == filename]
    if img_data.odc_x_rect.isna().values[0]:
        img_square, _, _ = make_square(img, THRESHOLD)
        img_384 = resize_square(img_square, 384)
        cv2.imwrite(f"data/ods/{filename}.jpg", img_384)
    else:
        # get deltas
        add_top = (img.shape[0] - img_data.side)/2
        add_left = (img.shape[1] - img_data.side)/2
        
        odc_x_rect = int(img_data.odc_x_square.values[0] + add_left)
        odc_y_rect = int(img_data.odc_y_square.values[0] + add_top)

        od = crop_od(img, odc_x_rect, odc_y_rect, int(img_data.odc_side_pxl.values[0]*FACTOR))
        od_384 = resize_square(od, 384)
        cv2.imwrite(f"data/ods/{img_data.new_file.values[0]}.jpg", od_384)


100%|██████████| 15000/15000 [45:46<00:00,  5.46it/s]  
